<a href="https://colab.research.google.com/github/Diviraage/-/blob/main/%D0%9F%D1%80%D0%B0%D0%BA%D1%82%D0%B8%D1%87%D0%BD%D0%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ПРАКТИЧНА РОБОТА
Аналіз рейтингу книг та жанрового покриття на основі датасету Goodreads

1. Мета роботи
Метою лабораторної роботи є набуття практичних навичок аналізу табличних даних у форматі CSV із використанням мови програмування Python та бібліотеки pandas, а також дослідження рейтингових характеристик книг і їх жанрового покриття на основі відкритого датасету Goodreads.

2. Вхідні дані та інструменти
Для виконання роботи використовується датасет goodbooks-10k, що містить інформацію про книги, користувацькі оцінки та жанрові теги. У ході роботи були задіяні такі файли:
books.csv — основна інформація про книги
ratings.csv — користувацькі оцінки
book_tags.csv — зв’язок книг із тегами
tags.csv — словник тегів


Аналіз проводився з використанням мови програмування Python та бібліотеки pandas.

**ЗАВАНТАЖЕННЯ ДАНИХ**

**3. Завантаження даних**
На першому етапі виконується зчитування CSV-файлів у пам’ять з подальшою перевіркою їх структури.Завантаження виконано успішно, структура таблиць відповідає документації датасету.

In [68]:
import pandas as pd
import os

file_names = {
    "books": "books.csv",
    "ratings": "ratings.csv",
    "book_tags": "book_tags.csv",
    "tags": "tags.csv"
}

dataframes = {}
all_files_exist = True

for df_name, file_name in file_names.items():
    if not os.path.exists(file_name):
        print(f"Warning: File '{file_name}' not found. Please ensure the dataset files are in the correct directory.")
        all_files_exist = False
        dataframes[df_name] = pd.DataFrame() # Create an empty DataFrame to prevent crashes
    else:
        try:
            dataframes[df_name] = pd.read_csv(file_name)
            print(f"File '{file_name}' loaded successfully.")
        except Exception as e:
            print(f"Error loading '{file_name}': {e}")
            all_files_exist = False
            dataframes[df_name] = pd.DataFrame() # Create an empty DataFrame if loading fails

books = dataframes.get("books")
ratings = dataframes.get("ratings")
book_tags = dataframes.get("book_tags")
tags = dataframes.get("tags")

if not all_files_exist:
    print("Some data files were not found or failed to load. Corresponding DataFrames might be empty.")
else:
    print("All specified data files were found and loaded successfully.")

File 'books.csv' loaded successfully.
File 'ratings.csv' loaded successfully.
File 'book_tags.csv' loaded successfully.
File 'tags.csv' loaded successfully.
All specified data files were found and loaded successfully.


**ПІДГОТОВКА ДАНИХ**

4. **Підготовка даних**
**4.1 Поєднання датасетів**
Для аналізу рейтингу необхідно поєднати дані про книги з інформацією про користувацькі оцінки. Об’єднання виконується за ідентифікатором книги book_id. У результаті формується набір даних, у якому кожна оцінка має відповідну інформацію про книгу.


In [69]:
data = ratings.merge(books, on="book_id", how="inner")

**4.2 Обчислення середнього рейтингу книг**
Для кожної книги було обчислено середній рейтинг та кількість отриманих оцінок. Цей етап дозволяє перейти від окремих оцінок до агрегованих характеристик книг.


In [70]:
book_stats = (
    ratings
    .groupby("book_id")
    .agg(
        mean_rating=("rating", "mean"),
        ratings_count=("rating", "count")
    )
    .reset_index()
)

books_with_stats = books.merge(book_stats, on="book_id", how="inner")
print("Columns in books_with_stats after merge (from i9lQ6nDve6_p):", books_with_stats.columns.tolist())

Columns in books_with_stats after merge (from i9lQ6nDve6_p): ['book_id', 'goodreads_book_id', 'best_book_id', 'work_id', 'books_count', 'isbn', 'isbn13', 'authors', 'original_publication_year', 'original_title', 'title', 'language_code', 'average_rating', 'ratings_count_x', 'work_ratings_count', 'work_text_reviews_count', 'ratings_1', 'ratings_2', 'ratings_3', 'ratings_4', 'ratings_5', 'image_url', 'small_image_url', 'mean_rating', 'ratings_count_y']


**4.3 Очищення даних від сміттєвого входу**
Для підвищення якості аналізу було виконано фільтрацію очевидно некоректних записів, зокрема книг з нульовими або від’ємними ідентифікаторами, відсутніми або нульовими рейтингами. Після очищення набір даних став придатним для подальшого статистичного аналізу.


In [71]:
print("Columns in books_with_stats before filtering (from C9fo8Pf9fDvh):", books_with_stats.columns.tolist())
books_clean = books_with_stats[
    (books_with_stats["book_id"] > 0) &
    (books_with_stats["ratings_count_y"] > 0) & # Corrected column name
    (books_with_stats["mean_rating"] > 0)
]

Columns in books_with_stats before filtering (from C9fo8Pf9fDvh): ['book_id', 'goodreads_book_id', 'best_book_id', 'work_id', 'books_count', 'isbn', 'isbn13', 'authors', 'original_publication_year', 'original_title', 'title', 'language_code', 'average_rating', 'ratings_count_x', 'work_ratings_count', 'work_text_reviews_count', 'ratings_1', 'ratings_2', 'ratings_3', 'ratings_4', 'ratings_5', 'image_url', 'small_image_url', 'mean_rating', 'ratings_count_y']


** Побудова рейтингу з порогом кількості оцінок** # Новий розділ

**5. Побудова рейтингу з порогом кількості оцінок**
Оскільки книги з малою кількістю оцінок можуть мати спотворений середній рейтинг, було введено поріг мінімальної кількості оцінок. У межах даної роботи використано поріг 100 оцінок.

In [72]:
threshold = 100

popular_books = books_clean[
    books_clean["ratings_count_y"] >= threshold # Corrected column name
]

На основі відфільтрованих даних було сформовано рейтинг книг за спаданням середнього рейтингу.

In [73]:
top_books = popular_books.sort_values(
    by="mean_rating",
    ascending=False
)

top_10_books = top_books.head(10)

Отриманий список представляє десять книг з найвищим середнім рейтингом за умови достатньої кількості оцінок.

**Аналіз жанрового покриття**# Новий розділ

**6. Аналіз жанрового покриття**
**6.1 Визначення жанрів книг**
Для визначення жанрів використано файли book_tags.csv та tags.csv, які було об’єднано між собою.


In [74]:
book_tags_full = book_tags.merge(tags, on="tag_id", how="inner")

Із загального набору тегів було відібрано жанрові теги, що найчастіше використовуються для класифікації книг.

In [75]:
genre_keywords = [
    "fantasy", "romance", "science fiction",
    "mystery", "thriller", "historical",
    "young adult"
]

genres = book_tags_full[
    book_tags_full["tag_name"].str.lower().isin(genre_keywords)
]

**6.2 Кількість жанрів для кожної книги**
Для кожної книги було визначено кількість унікальних жанрів, які вона охоплює. Цей показник дозволяє оцінити жанрову різноманітність книг.


In [76]:
genres_per_book = (
    genres
    .groupby("goodreads_book_id")["tag_name"]
    .nunique()
    .reset_index(name="genres_count")
)

**6.3 Найпопулярніші жанри у топ-10 книг**
Для книг із топ-10 рейтингу було проаналізовано частоту жанрів.
Отримані результати свідчать про домінування жанрів fantasy, romance та science fiction серед найрейтинговіших книг.

In [77]:
top_genres = genres[
    genres["goodreads_book_id"].isin(top_10_books["book_id"])
]

top_genre_counts = top_genres["tag_name"].value_counts().head(10)


**7. Висновки**
У ході виконання лабораторної роботи було проведено повний цикл аналізу табличних даних на основі датасету Goodreads. У процесі виконання цієї лабораторної роботи я навчився працювати з реальним датасетом у форматі CSV, поєднувати кілька таблиць та очищати дані від некоректних значень, що є важливим етапом будь-якого аналізу. Розрахунок середнього рейтингу книг із урахуванням мінімальної кількості оцінок показав, що такий підхід дозволяє отримати більш об’єктивний та надійний рейтинг, ніж використання всіх даних без фільтрації. Під час аналізу жанрового покриття стало помітно, що найпопулярніші книги часто належать одразу до кількох жанрів, а найбільш поширеними серед них є fantasy, romance та science fiction. Отримані результати дають розуміння того, як жанри та кількість оцінок впливають на рейтинг книг, і можуть бути корисними для подальших досліджень або побудови рекомендаційних систем.
